In [1]:
import torch


from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F

import numpy as np

import matplotlib.pyplot as plt
#from sklearn.manifold import TSNE

#import math

#import gc

from utils import *

from sklearn.preprocessing import MinMaxScaler

from scipy.stats import pearsonr

import seaborn as sns
import os
import scipy
import scipy.io

In [2]:
cuda = True if torch.cuda.is_available() else False

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

device = torch.device("cuda:0" if cuda else "cpu")
#device = 'cpu'
print("Device")
print(device)

Device
cuda:0


In [3]:
a = scipy.io.loadmat("../data/zeisel/zeisel_data.mat")
data= a['zeisel_data'].T
N,d=data.shape

#load labels (first level of the hierarchy) from file
a = scipy.io.loadmat("../data/zeisel/zeisel_labels1.mat")
l_aux = a['zeisel_labels1']
l_0=[l_aux[i][0] for i in range(l_aux.shape[0])]
#load labels (second level of the hierarchy) from file
a = scipy.io.loadmat("../data/zeisel/zeisel_labels2.mat")
l_aux = a['zeisel_labels2']
l_1=[l_aux[i][0] for i in range(l_aux.shape[0])]
#construct an array with hierarchy labels
labels=np.array([l_0, l_1])

# load names from file 
a = scipy.io.loadmat("../data/zeisel/zeisel_names.mat")
names0=[a['zeisel_names'][i][0][0] for i in range(N)]
names1=[a['zeisel_names'][i][1][0] for i in range(N)]


slices = np.random.permutation(np.arange(data.shape[0]))
upto = int(.8 * len(data))

train_data = data[slices[:upto]]
test_data = data[slices[upto:]]



scaler = MinMaxScaler()
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

train_data = Tensor(train_data).to(device)
test_data = Tensor(test_data).to(device)

In [4]:
N = 10000
z_size = 100

# really good results for vanilla VAE on synthetic data with EPOCHS set to 50, 
# but when running locally set to 10 for reasonable run times
n_epochs = 600
batch_size = 64
lr = 0.0001
b1 = 0.9
b2 = 0.999

global_t = 4
k = 50

In [5]:
def train_model(train_data, model):
    optimizer = torch.optim.Adam(model.parameters(), 
                                 lr=lr, 
                                 betas = (b1,b2))
        
    for epoch in range(1, n_epochs+1):
        train(train_data, 
              model, 
              optimizer, 
              epoch, 
              batch_size)
        model.t = max(0.001, model.t * 0.99)

        
    return model

def save_model(base_path, model):
    # make directory
    if not os.path.exists(os.path.dirname(base_path)):
        try:
            os.makedirs(os.path.dirname(base_path))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise Exception("COULD NOT MAKE PATH")
    with open(base_path, 'wb') as PATH:
        torch.save(model.state_dict(), PATH)

In [6]:
def top_logits_gumbel_runningstate_vae(data, model):
    assert isinstance(model, VAE_Gumbel_RunningState)
    with torch.no_grad():
        w = model.logit_enc.clone().view(-1)
        top_k_logits = torch.topk(w, k = model.k, sorted = True)[1]
        enc_top_logits = torch.nn.functional.one_hot(top_k_logits, num_classes = data.shape[1]).sum(dim = 0)
        
        #subsets = sample_subset(w, model.k,model.t,True)
        subsets = sample_subset(w, model.k,model.t)
        #max_idx = torch.argmax(subsets, 1, keepdim=True)
        #one_hot = Tensor(subsets.shape)
        #one_hot.zero_()
        #one_hot.scatter_(1, max_idx, 1)

        
    return enc_top_logits, subsets

def top_logits_gumbel_concrete_vae_nsml(data, model):
    assert isinstance(model, ConcreteVAE_NMSL)
    
    with torch.no_grad():

        w = gumbel_keys(model.logit_enc, EPSILON = torch.finfo(torch.float32).eps)
        w = torch.softmax(w/model.t, dim = -1)
        subset_indices = w.clone().detach()

        #max_idx = torch.argmax(subset_indices, 1, keepdim=True)
        #one_hot = Tensor(subset_indices.shape)
        #one_hot.zero_()
        #one_hot.scatter_(1, max_idx, 1)

        all_subsets = subset_indices.sum(dim = 0)

        inds = torch.argsort(subset_indices.sum(dim = 0), descending = True)[:model.k]
        all_logits = torch.nn.functional.one_hot(inds, num_classes = data.shape[1]).sum(dim = 0)
        
        
        
        
    return all_logits, all_subsets

In [7]:
model = VAE_Gumbel_RunningState(train_data.shape[1], 200, 50, k = k, t = global_t, alpha = 0.9)
model.to(device)
train_model(train_data, model)
model.set_burned_in()

Train Epoch: 1 [0/2404 (0%)]	Loss: 2804.311279
Train Epoch: 1 [1280/2404 (53%)]	Loss: 2714.453125
====> Epoch: 1 Average loss: 2719.7863
Train Epoch: 2 [0/2404 (0%)]	Loss: 2645.707520
Train Epoch: 2 [1280/2404 (53%)]	Loss: 2559.585449
====> Epoch: 2 Average loss: 2562.4391
Train Epoch: 3 [0/2404 (0%)]	Loss: 2474.410645
Train Epoch: 3 [1280/2404 (53%)]	Loss: 2389.212891
====> Epoch: 3 Average loss: 2402.9005
Train Epoch: 4 [0/2404 (0%)]	Loss: 2268.050781
Train Epoch: 4 [1280/2404 (53%)]	Loss: 2294.530029
====> Epoch: 4 Average loss: 2254.6724
Train Epoch: 5 [0/2404 (0%)]	Loss: 2175.658447
Train Epoch: 5 [1280/2404 (53%)]	Loss: 2236.860107
====> Epoch: 5 Average loss: 2180.2413
Train Epoch: 6 [0/2404 (0%)]	Loss: 2104.043457
Train Epoch: 6 [1280/2404 (53%)]	Loss: 2194.937012
====> Epoch: 6 Average loss: 2136.3696
Train Epoch: 7 [0/2404 (0%)]	Loss: 2110.184326
Train Epoch: 7 [1280/2404 (53%)]	Loss: 2136.951660
====> Epoch: 7 Average loss: 2119.3101
Train Epoch: 8 [0/2404 (0%)]	Loss: 2220.9

====> Epoch: 59 Average loss: 2024.0044
Train Epoch: 60 [0/2404 (0%)]	Loss: 1974.234009
Train Epoch: 60 [1280/2404 (53%)]	Loss: 1912.356567
====> Epoch: 60 Average loss: 2023.7686
Train Epoch: 61 [0/2404 (0%)]	Loss: 2102.445068
Train Epoch: 61 [1280/2404 (53%)]	Loss: 2101.110107
====> Epoch: 61 Average loss: 2022.3812
Train Epoch: 62 [0/2404 (0%)]	Loss: 2065.404053
Train Epoch: 62 [1280/2404 (53%)]	Loss: 1978.165161
====> Epoch: 62 Average loss: 2022.2131
Train Epoch: 63 [0/2404 (0%)]	Loss: 1967.840820
Train Epoch: 63 [1280/2404 (53%)]	Loss: 2039.783691
====> Epoch: 63 Average loss: 2020.9206
Train Epoch: 64 [0/2404 (0%)]	Loss: 2029.067871
Train Epoch: 64 [1280/2404 (53%)]	Loss: 1986.098877
====> Epoch: 64 Average loss: 2018.8638
Train Epoch: 65 [0/2404 (0%)]	Loss: 1918.427612
Train Epoch: 65 [1280/2404 (53%)]	Loss: 1905.456421
====> Epoch: 65 Average loss: 2021.4104
Train Epoch: 66 [0/2404 (0%)]	Loss: 1932.628052
Train Epoch: 66 [1280/2404 (53%)]	Loss: 2010.321899
====> Epoch: 66 Aver

Train Epoch: 118 [1280/2404 (53%)]	Loss: 2146.382812
====> Epoch: 118 Average loss: 2024.6919
Train Epoch: 119 [0/2404 (0%)]	Loss: 1905.668457
Train Epoch: 119 [1280/2404 (53%)]	Loss: 2010.432129
====> Epoch: 119 Average loss: 2030.8778
Train Epoch: 120 [0/2404 (0%)]	Loss: 1917.799438
Train Epoch: 120 [1280/2404 (53%)]	Loss: 2084.218262
====> Epoch: 120 Average loss: 2023.0372
Train Epoch: 121 [0/2404 (0%)]	Loss: 2000.787231
Train Epoch: 121 [1280/2404 (53%)]	Loss: 1944.815552
====> Epoch: 121 Average loss: 2022.1427
Train Epoch: 122 [0/2404 (0%)]	Loss: 1952.120972
Train Epoch: 122 [1280/2404 (53%)]	Loss: 2076.024414
====> Epoch: 122 Average loss: 2028.8640
Train Epoch: 123 [0/2404 (0%)]	Loss: 1991.797485
Train Epoch: 123 [1280/2404 (53%)]	Loss: 2225.925049
====> Epoch: 123 Average loss: 2033.1398
Train Epoch: 124 [0/2404 (0%)]	Loss: 2040.827637
Train Epoch: 124 [1280/2404 (53%)]	Loss: 2011.490479
====> Epoch: 124 Average loss: 2025.4590
Train Epoch: 125 [0/2404 (0%)]	Loss: 2069.673340

====> Epoch: 175 Average loss: 2023.3863
Train Epoch: 176 [0/2404 (0%)]	Loss: 1955.934937
Train Epoch: 176 [1280/2404 (53%)]	Loss: 1932.899536
====> Epoch: 176 Average loss: 2022.9597
Train Epoch: 177 [0/2404 (0%)]	Loss: 2036.673828
Train Epoch: 177 [1280/2404 (53%)]	Loss: 2115.219727
====> Epoch: 177 Average loss: 2023.5643
Train Epoch: 178 [0/2404 (0%)]	Loss: 1999.889648
Train Epoch: 178 [1280/2404 (53%)]	Loss: 2055.830322
====> Epoch: 178 Average loss: 2022.0133
Train Epoch: 179 [0/2404 (0%)]	Loss: 1967.518921
Train Epoch: 179 [1280/2404 (53%)]	Loss: 1995.635864
====> Epoch: 179 Average loss: 2024.3357
Train Epoch: 180 [0/2404 (0%)]	Loss: 2109.194824
Train Epoch: 180 [1280/2404 (53%)]	Loss: 1978.545776
====> Epoch: 180 Average loss: 2020.3996
Train Epoch: 181 [0/2404 (0%)]	Loss: 2054.590820
Train Epoch: 181 [1280/2404 (53%)]	Loss: 2022.087524
====> Epoch: 181 Average loss: 2021.7470
Train Epoch: 182 [0/2404 (0%)]	Loss: 2078.234131
Train Epoch: 182 [1280/2404 (53%)]	Loss: 2072.211670

Train Epoch: 233 [1280/2404 (53%)]	Loss: 2046.998047
====> Epoch: 233 Average loss: 1992.3626
Train Epoch: 234 [0/2404 (0%)]	Loss: 2021.209595
Train Epoch: 234 [1280/2404 (53%)]	Loss: 2122.466064
====> Epoch: 234 Average loss: 1995.3649
Train Epoch: 235 [0/2404 (0%)]	Loss: 2030.583984
Train Epoch: 235 [1280/2404 (53%)]	Loss: 1987.159668
====> Epoch: 235 Average loss: 1996.6368
Train Epoch: 236 [0/2404 (0%)]	Loss: 1930.820923
Train Epoch: 236 [1280/2404 (53%)]	Loss: 2096.398193
====> Epoch: 236 Average loss: 1997.7052
Train Epoch: 237 [0/2404 (0%)]	Loss: 1933.134033
Train Epoch: 237 [1280/2404 (53%)]	Loss: 1985.730835
====> Epoch: 237 Average loss: 1991.4645
Train Epoch: 238 [0/2404 (0%)]	Loss: 1872.728027
Train Epoch: 238 [1280/2404 (53%)]	Loss: 1984.906250
====> Epoch: 238 Average loss: 1987.8746
Train Epoch: 239 [0/2404 (0%)]	Loss: 1965.606934
Train Epoch: 239 [1280/2404 (53%)]	Loss: 1924.301025
====> Epoch: 239 Average loss: 1989.0517
Train Epoch: 240 [0/2404 (0%)]	Loss: 2018.361816

...

Train Epoch: 348 [1280/2404 (53%)]	Loss: 1990.723877
====> Epoch: 348 Average loss: 1942.4646
Train Epoch: 349 [0/2404 (0%)]	Loss: 1972.689331
Train Epoch: 349 [1280/2404 (53%)]	Loss: 1933.291260
====> Epoch: 349 Average loss: 1938.7805
Train Epoch: 350 [0/2404 (0%)]	Loss: 1885.401245
Train Epoch: 350 [1280/2404 (53%)]	Loss: 1949.783813
====> Epoch: 350 Average loss: 1940.9146
Train Epoch: 351 [0/2404 (0%)]	Loss: 1896.022949
Train Epoch: 351 [1280/2404 (53%)]	Loss: 1938.162231
====> Epoch: 351 Average loss: 1938.8266
Train Epoch: 352 [0/2404 (0%)]	Loss: 2016.717041
Train Epoch: 352 [1280/2404 (53%)]	Loss: 1937.672119
====> Epoch: 352 Average loss: 1943.1686
Train Epoch: 353 [0/2404 (0%)]	Loss: 1850.750977
Train Epoch: 353 [1280/2404 (53%)]	Loss: 1943.760864
====> Epoch: 353 Average loss: 1938.1879
Train Epoch: 354 [0/2404 (0%)]	Loss: 1953.719971
Train Epoch: 354 [1280/2404 (53%)]	Loss: 1949.308350
====> Epoch: 354 Average loss: 1942.4045
Train Epoch: 355 [0/2404 (0%)]	Loss: 1792.523438

====> Epoch: 405 Average loss: 1910.5306
Train Epoch: 406 [0/2404 (0%)]	Loss: 1872.268921
Train Epoch: 406 [1280/2404 (53%)]	Loss: 1791.669800
====> Epoch: 406 Average loss: 1906.5771
Train Epoch: 407 [0/2404 (0%)]	Loss: 1878.049316
Train Epoch: 407 [1280/2404 (53%)]	Loss: 1877.176025
====> Epoch: 407 Average loss: 1908.4568
Train Epoch: 408 [0/2404 (0%)]	Loss: 1789.333984
Train Epoch: 408 [1280/2404 (53%)]	Loss: 1967.635864
====> Epoch: 408 Average loss: 1910.9125
Train Epoch: 409 [0/2404 (0%)]	Loss: 1966.830444
Train Epoch: 409 [1280/2404 (53%)]	Loss: 1945.663818
====> Epoch: 409 Average loss: 1907.5444
Train Epoch: 410 [0/2404 (0%)]	Loss: 1965.453369
Train Epoch: 410 [1280/2404 (53%)]	Loss: 1715.757080
====> Epoch: 410 Average loss: 1910.4051
Train Epoch: 411 [0/2404 (0%)]	Loss: 1806.088501
Train Epoch: 411 [1280/2404 (53%)]	Loss: 1893.570923
====> Epoch: 411 Average loss: 1907.8604
Train Epoch: 412 [0/2404 (0%)]	Loss: 1904.226807
Train Epoch: 412 [1280/2404 (53%)]	Loss: 1925.018433

Train Epoch: 463 [1280/2404 (53%)]	Loss: 1899.883423
====> Epoch: 463 Average loss: 1893.9608
Train Epoch: 464 [0/2404 (0%)]	Loss: 1830.419189
Train Epoch: 464 [1280/2404 (53%)]	Loss: 1759.691406
====> Epoch: 464 Average loss: 1895.7253
Train Epoch: 465 [0/2404 (0%)]	Loss: 1752.343506
Train Epoch: 465 [1280/2404 (53%)]	Loss: 2030.550781
====> Epoch: 465 Average loss: 1895.2169
Train Epoch: 466 [0/2404 (0%)]	Loss: 2091.399658
Train Epoch: 466 [1280/2404 (53%)]	Loss: 1731.890503
====> Epoch: 466 Average loss: 1893.7589
Train Epoch: 467 [0/2404 (0%)]	Loss: 1908.092651
Train Epoch: 467 [1280/2404 (53%)]	Loss: 1948.208618
====> Epoch: 467 Average loss: 1895.2960
Train Epoch: 468 [0/2404 (0%)]	Loss: 1873.302612
Train Epoch: 468 [1280/2404 (53%)]	Loss: 1816.252563
====> Epoch: 468 Average loss: 1894.5266
Train Epoch: 469 [0/2404 (0%)]	Loss: 1829.445068
Train Epoch: 469 [1280/2404 (53%)]	Loss: 1782.157959
====> Epoch: 469 Average loss: 1894.5988
Train Epoch: 470 [0/2404 (0%)]	Loss: 1935.957520

====> Epoch: 520 Average loss: 1886.4293
Train Epoch: 521 [0/2404 (0%)]	Loss: 1907.622559
Train Epoch: 521 [1280/2404 (53%)]	Loss: 1919.870972
====> Epoch: 521 Average loss: 1885.6462
Train Epoch: 522 [0/2404 (0%)]	Loss: 1936.306763
Train Epoch: 522 [1280/2404 (53%)]	Loss: 1875.089600
====> Epoch: 522 Average loss: 1884.4226
Train Epoch: 523 [0/2404 (0%)]	Loss: 1952.063965
Train Epoch: 523 [1280/2404 (53%)]	Loss: 1898.705200
====> Epoch: 523 Average loss: 1886.8171
Train Epoch: 524 [0/2404 (0%)]	Loss: 1809.775269
Train Epoch: 524 [1280/2404 (53%)]	Loss: 1739.879883
====> Epoch: 524 Average loss: 1885.1492
Train Epoch: 525 [0/2404 (0%)]	Loss: 1824.685913
Train Epoch: 525 [1280/2404 (53%)]	Loss: 1988.421753
====> Epoch: 525 Average loss: 1884.8867
Train Epoch: 526 [0/2404 (0%)]	Loss: 1932.923950
Train Epoch: 526 [1280/2404 (53%)]	Loss: 1951.040649
====> Epoch: 526 Average loss: 1883.9219
Train Epoch: 527 [0/2404 (0%)]	Loss: 1922.262085
Train Epoch: 527 [1280/2404 (53%)]	Loss: 1874.329224

Train Epoch: 578 [1280/2404 (53%)]	Loss: 1831.498169
====> Epoch: 578 Average loss: 1879.1089
Train Epoch: 579 [0/2404 (0%)]	Loss: 1840.593384
Train Epoch: 579 [1280/2404 (53%)]	Loss: 1862.584717
====> Epoch: 579 Average loss: 1876.6190
Train Epoch: 580 [0/2404 (0%)]	Loss: 1859.634766
Train Epoch: 580 [1280/2404 (53%)]	Loss: 1829.986816
====> Epoch: 580 Average loss: 1878.4924
Train Epoch: 581 [0/2404 (0%)]	Loss: 1923.599976
Train Epoch: 581 [1280/2404 (53%)]	Loss: 1913.214722
====> Epoch: 581 Average loss: 1879.7926
Train Epoch: 582 [0/2404 (0%)]	Loss: 1927.076782
Train Epoch: 582 [1280/2404 (53%)]	Loss: 1834.333618
====> Epoch: 582 Average loss: 1878.5590
Train Epoch: 583 [0/2404 (0%)]	Loss: 1861.788940
Train Epoch: 583 [1280/2404 (53%)]	Loss: 1751.349731
====> Epoch: 583 Average loss: 1878.5802
Train Epoch: 584 [0/2404 (0%)]	Loss: 1901.239136
Train Epoch: 584 [1280/2404 (53%)]	Loss: 1903.656006
====> Epoch: 584 Average loss: 1877.2785
Train Epoch: 585 [0/2404 (0%)]	Loss: 1982.682617

In [8]:
top_logits_running_state = top_logits_gumbel_runningstate_vae(test_data, model)

In [9]:
torch.argsort(top_logits_running_state[0], descending = True)[:k]

tensor([   0,    2,    4,    7,   13,   17,   26,   36,   51,   63,   64,   90,
          92,  104,  106,  133,  138,  146,  150,  154,  173,  213,  245,  317,
         388,  421,  427,  448,  480,  601,  612,  826,  839,  851, 1029, 1079,
        1124, 1563, 1747, 2229, 2392, 2397, 2538, 2746, 2756, 2784, 2855, 2866,
        3119, 3521], device='cuda:0')

In [10]:
inds = torch.argsort(top_logits_running_state[0], descending = True)[:50].cpu().numpy()

In [11]:
len(labels[0])
print("HOW TO GET NAME OF FEATURES?")

HOW TO GET NAME OF FEATURES?


In [12]:
save_model("../data/models/final_run_zeisel/runningstate_vae/k_50/model.pt", model)

In [13]:
model = ConcreteVAE_NMSL(train_data.shape[1], 200, 50, k = k, t = global_t)
model.to(device)
train_model(train_data, model)

Train Epoch: 1 [0/2404 (0%)]	Loss: 2798.588379
Train Epoch: 1 [1280/2404 (53%)]	Loss: 2706.293457
====> Epoch: 1 Average loss: 2714.6364
Train Epoch: 2 [0/2404 (0%)]	Loss: 2627.192139
Train Epoch: 2 [1280/2404 (53%)]	Loss: 2545.523193
====> Epoch: 2 Average loss: 2549.6524
Train Epoch: 3 [0/2404 (0%)]	Loss: 2462.881348
Train Epoch: 3 [1280/2404 (53%)]	Loss: 2333.375488
====> Epoch: 3 Average loss: 2388.7925
Train Epoch: 4 [0/2404 (0%)]	Loss: 2323.447998
Train Epoch: 4 [1280/2404 (53%)]	Loss: 2262.056885
====> Epoch: 4 Average loss: 2282.6377
Train Epoch: 5 [0/2404 (0%)]	Loss: 2293.728271
Train Epoch: 5 [1280/2404 (53%)]	Loss: 2242.334717
====> Epoch: 5 Average loss: 2222.3463
Train Epoch: 6 [0/2404 (0%)]	Loss: 2257.388672
Train Epoch: 6 [1280/2404 (53%)]	Loss: 2240.429688
====> Epoch: 6 Average loss: 2166.5590
Train Epoch: 7 [0/2404 (0%)]	Loss: 2196.955322
Train Epoch: 7 [1280/2404 (53%)]	Loss: 1986.762573
====> Epoch: 7 Average loss: 2107.2921
Train Epoch: 8 [0/2404 (0%)]	Loss: 2175.0

Train Epoch: 60 [1280/2404 (53%)]	Loss: 1792.183960
====> Epoch: 60 Average loss: 1921.0144
Train Epoch: 61 [0/2404 (0%)]	Loss: 1931.370850
Train Epoch: 61 [1280/2404 (53%)]	Loss: 2027.370850
====> Epoch: 61 Average loss: 1920.9372
Train Epoch: 62 [0/2404 (0%)]	Loss: 2021.890747
Train Epoch: 62 [1280/2404 (53%)]	Loss: 2072.137451
====> Epoch: 62 Average loss: 1920.9568
Train Epoch: 63 [0/2404 (0%)]	Loss: 1763.636108
Train Epoch: 63 [1280/2404 (53%)]	Loss: 1877.632690
====> Epoch: 63 Average loss: 1920.8284
Train Epoch: 64 [0/2404 (0%)]	Loss: 1940.229980
Train Epoch: 64 [1280/2404 (53%)]	Loss: 1905.826416
====> Epoch: 64 Average loss: 1920.7421
Train Epoch: 65 [0/2404 (0%)]	Loss: 1806.717285
Train Epoch: 65 [1280/2404 (53%)]	Loss: 1965.922363
====> Epoch: 65 Average loss: 1920.5400
Train Epoch: 66 [0/2404 (0%)]	Loss: 1903.662964
Train Epoch: 66 [1280/2404 (53%)]	Loss: 1965.881714
====> Epoch: 66 Average loss: 1920.5512
Train Epoch: 67 [0/2404 (0%)]	Loss: 1799.053467
Train Epoch: 67 [128

Train Epoch: 119 [1280/2404 (53%)]	Loss: 1929.474854
====> Epoch: 119 Average loss: 1914.1710
Train Epoch: 120 [0/2404 (0%)]	Loss: 1994.694580
Train Epoch: 120 [1280/2404 (53%)]	Loss: 1933.774048
====> Epoch: 120 Average loss: 1914.0002
Train Epoch: 121 [0/2404 (0%)]	Loss: 1963.504517
Train Epoch: 121 [1280/2404 (53%)]	Loss: 1841.436646
====> Epoch: 121 Average loss: 1913.8724
Train Epoch: 122 [0/2404 (0%)]	Loss: 1895.655884
Train Epoch: 122 [1280/2404 (53%)]	Loss: 1942.021362
====> Epoch: 122 Average loss: 1913.6888
Train Epoch: 123 [0/2404 (0%)]	Loss: 1955.434570
Train Epoch: 123 [1280/2404 (53%)]	Loss: 2080.579590
====> Epoch: 123 Average loss: 1913.5349
Train Epoch: 124 [0/2404 (0%)]	Loss: 1915.688599
Train Epoch: 124 [1280/2404 (53%)]	Loss: 1972.586670
====> Epoch: 124 Average loss: 1913.3521
Train Epoch: 125 [0/2404 (0%)]	Loss: 1872.650513
Train Epoch: 125 [1280/2404 (53%)]	Loss: 1974.800537
====> Epoch: 125 Average loss: 1913.6918
Train Epoch: 126 [0/2404 (0%)]	Loss: 1956.560791

Train Epoch: 177 [1280/2404 (53%)]	Loss: 1974.275513
====> Epoch: 177 Average loss: 1888.5724
Train Epoch: 178 [0/2404 (0%)]	Loss: 1989.241943
Train Epoch: 178 [1280/2404 (53%)]	Loss: 1913.437988
====> Epoch: 178 Average loss: 1887.0717
Train Epoch: 179 [0/2404 (0%)]	Loss: 1800.275024
Train Epoch: 179 [1280/2404 (53%)]	Loss: 1922.959229
====> Epoch: 179 Average loss: 1886.4753
Train Epoch: 180 [0/2404 (0%)]	Loss: 1907.935913
Train Epoch: 180 [1280/2404 (53%)]	Loss: 1930.676514
====> Epoch: 180 Average loss: 1887.0190
Train Epoch: 181 [0/2404 (0%)]	Loss: 1717.129761
Train Epoch: 181 [1280/2404 (53%)]	Loss: 1862.221069
====> Epoch: 181 Average loss: 1885.8686
Train Epoch: 182 [0/2404 (0%)]	Loss: 1964.240723
Train Epoch: 182 [1280/2404 (53%)]	Loss: 1868.575684
====> Epoch: 182 Average loss: 1886.1201
Train Epoch: 183 [0/2404 (0%)]	Loss: 2061.714355
Train Epoch: 183 [1280/2404 (53%)]	Loss: 1889.424438
====> Epoch: 183 Average loss: 1885.3424
Train Epoch: 184 [0/2404 (0%)]	Loss: 1834.870605

Train Epoch: 235 [1280/2404 (53%)]	Loss: 1973.663818
====> Epoch: 235 Average loss: 1881.1621
Train Epoch: 236 [0/2404 (0%)]	Loss: 1772.169434
Train Epoch: 236 [1280/2404 (53%)]	Loss: 1835.000854
====> Epoch: 236 Average loss: 1882.1228
Train Epoch: 237 [0/2404 (0%)]	Loss: 1893.811768
Train Epoch: 237 [1280/2404 (53%)]	Loss: 1899.459717
====> Epoch: 237 Average loss: 1882.0465
Train Epoch: 238 [0/2404 (0%)]	Loss: 2006.815063
Train Epoch: 238 [1280/2404 (53%)]	Loss: 1814.229736
====> Epoch: 238 Average loss: 1882.0293
Train Epoch: 239 [0/2404 (0%)]	Loss: 1969.807129
Train Epoch: 239 [1280/2404 (53%)]	Loss: 1802.807983
====> Epoch: 239 Average loss: 1881.7593
Train Epoch: 240 [0/2404 (0%)]	Loss: 1825.421387
Train Epoch: 240 [1280/2404 (53%)]	Loss: 1855.260132
====> Epoch: 240 Average loss: 1881.8700
Train Epoch: 241 [0/2404 (0%)]	Loss: 1816.098755
Train Epoch: 241 [1280/2404 (53%)]	Loss: 1885.992432
====> Epoch: 241 Average loss: 1882.7124
Train Epoch: 242 [0/2404 (0%)]	Loss: 1912.678467

...

Train Epoch: 408 [1280/2404 (53%)]	Loss: 1880.713379
====> Epoch: 408 Average loss: 1889.9648
Train Epoch: 409 [0/2404 (0%)]	Loss: 1879.953003
Train Epoch: 409 [1280/2404 (53%)]	Loss: 1942.889404
====> Epoch: 409 Average loss: 1888.4854
Train Epoch: 410 [0/2404 (0%)]	Loss: 1886.026489
Train Epoch: 410 [1280/2404 (53%)]	Loss: 1785.851196
====> Epoch: 410 Average loss: 1890.1419
Train Epoch: 411 [0/2404 (0%)]	Loss: 1935.597168
Train Epoch: 411 [1280/2404 (53%)]	Loss: 1771.429321
====> Epoch: 411 Average loss: 1890.2389
Train Epoch: 412 [0/2404 (0%)]	Loss: 1995.331421
Train Epoch: 412 [1280/2404 (53%)]	Loss: 1772.373657
====> Epoch: 412 Average loss: 1889.6099
Train Epoch: 413 [0/2404 (0%)]	Loss: 1849.553711
Train Epoch: 413 [1280/2404 (53%)]	Loss: 1927.180542
====> Epoch: 413 Average loss: 1889.3492
Train Epoch: 414 [0/2404 (0%)]	Loss: 1890.622803
Train Epoch: 414 [1280/2404 (53%)]	Loss: 1857.317139
====> Epoch: 414 Average loss: 1890.6933
Train Epoch: 415 [0/2404 (0%)]	Loss: 1808.766357

Train Epoch: 466 [1280/2404 (53%)]	Loss: 1890.222534
====> Epoch: 466 Average loss: 1889.2105
Train Epoch: 467 [0/2404 (0%)]	Loss: 1955.407227
Train Epoch: 467 [1280/2404 (53%)]	Loss: 1916.142090
====> Epoch: 467 Average loss: 1891.3147
Train Epoch: 468 [0/2404 (0%)]	Loss: 1872.409424
Train Epoch: 468 [1280/2404 (53%)]	Loss: 1947.341797
====> Epoch: 468 Average loss: 1890.2687
Train Epoch: 469 [0/2404 (0%)]	Loss: 1859.011108
Train Epoch: 469 [1280/2404 (53%)]	Loss: 1866.359253
====> Epoch: 469 Average loss: 1890.2704
Train Epoch: 470 [0/2404 (0%)]	Loss: 1776.725708
Train Epoch: 470 [1280/2404 (53%)]	Loss: 1822.196533
====> Epoch: 470 Average loss: 1890.2404
Train Epoch: 471 [0/2404 (0%)]	Loss: 1910.813354
Train Epoch: 471 [1280/2404 (53%)]	Loss: 1848.687988
====> Epoch: 471 Average loss: 1888.7441
Train Epoch: 472 [0/2404 (0%)]	Loss: 1969.763794
Train Epoch: 472 [1280/2404 (53%)]	Loss: 1902.699829
====> Epoch: 472 Average loss: 1890.3162
Train Epoch: 473 [0/2404 (0%)]	Loss: 1727.639771

Train Epoch: 524 [1280/2404 (53%)]	Loss: 1792.190552
====> Epoch: 524 Average loss: 1890.0656
Train Epoch: 525 [0/2404 (0%)]	Loss: 1846.029419
Train Epoch: 525 [1280/2404 (53%)]	Loss: 1862.751831
====> Epoch: 525 Average loss: 1887.8578
Train Epoch: 526 [0/2404 (0%)]	Loss: 1870.898926
Train Epoch: 526 [1280/2404 (53%)]	Loss: 1868.302612
====> Epoch: 526 Average loss: 1889.2596
Train Epoch: 527 [0/2404 (0%)]	Loss: 1867.530151
Train Epoch: 527 [1280/2404 (53%)]	Loss: 1892.919067
====> Epoch: 527 Average loss: 1889.6063
Train Epoch: 528 [0/2404 (0%)]	Loss: 1845.114380
Train Epoch: 528 [1280/2404 (53%)]	Loss: 1910.042480
====> Epoch: 528 Average loss: 1890.3909
Train Epoch: 529 [0/2404 (0%)]	Loss: 1924.682617
Train Epoch: 529 [1280/2404 (53%)]	Loss: 1954.467529
====> Epoch: 529 Average loss: 1890.7394
Train Epoch: 530 [0/2404 (0%)]	Loss: 1888.717651
Train Epoch: 530 [1280/2404 (53%)]	Loss: 1918.167358
====> Epoch: 530 Average loss: 1890.8248
Train Epoch: 531 [0/2404 (0%)]	Loss: 1733.926392

====> Epoch: 581 Average loss: 1890.1977
Train Epoch: 582 [0/2404 (0%)]	Loss: 1910.989136
Train Epoch: 582 [1280/2404 (53%)]	Loss: 1825.785645
====> Epoch: 582 Average loss: 1890.7496
Train Epoch: 583 [0/2404 (0%)]	Loss: 1882.695679
Train Epoch: 583 [1280/2404 (53%)]	Loss: 1873.035156
====> Epoch: 583 Average loss: 1889.5106
Train Epoch: 584 [0/2404 (0%)]	Loss: 1868.418335
Train Epoch: 584 [1280/2404 (53%)]	Loss: 1931.505371
====> Epoch: 584 Average loss: 1890.9907
Train Epoch: 585 [0/2404 (0%)]	Loss: 1963.414429
Train Epoch: 585 [1280/2404 (53%)]	Loss: 1905.063477
====> Epoch: 585 Average loss: 1888.4598
Train Epoch: 586 [0/2404 (0%)]	Loss: 1847.620972
Train Epoch: 586 [1280/2404 (53%)]	Loss: 1959.969360
====> Epoch: 586 Average loss: 1890.0474
Train Epoch: 587 [0/2404 (0%)]	Loss: 1872.460571
Train Epoch: 587 [1280/2404 (53%)]	Loss: 1837.467285
====> Epoch: 587 Average loss: 1888.5036
Train Epoch: 588 [0/2404 (0%)]	Loss: 1860.981567
Train Epoch: 588 [1280/2404 (53%)]	Loss: 1977.565552

ConcreteVAE_NMSL(
  (encoder): Sequential(
    (0): Linear(in_features=50, out_features=200, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
  )
  (enc_mean): Linear(in_features=200, out_features=50, bias=True)
  (enc_logvar): Linear(in_features=200, out_features=50, bias=True)
  (decoder): Sequential(
    (0): Linear(in_features=50, out_features=200, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=200, out_features=4000, bias=True)
    (3): Sigmoid()
  )
)

In [14]:
top_logits_concrete = top_logits_gumbel_concrete_vae_nsml(test_data, model)

In [15]:
torch.argsort(top_logits_concrete[0], descending = True)[:k]

tensor([  20,  147,  148,  263,  266,  464,  515,  624,  627,  662,  718,  739,
         805,  855,  917,  945, 1001, 1061, 1155, 1187, 1189, 1205, 1220, 1243,
        1261, 1317, 1418, 1491, 1718, 1790, 1870, 1956, 1961, 2034, 2071, 2098,
        2139, 2187, 2230, 2362, 2547, 2709, 2747, 2805, 2834, 3248, 3333, 3342,
        3586, 3715], device='cuda:0')

In [16]:
save_model("../data/models/final_run_zeisel/concrete_vae/k_50/model.pt", model)